## Installation

We will need to install a number of libraries to start with.

In [2]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

We use flan alpaca model for speed and local execution.

In [3]:
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
#model = pipeline("text-generation", model="bigscience/bloom-1b7")
#model = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")
#model = pipeline(model="declare-lab/flan-alpaca-xl", device=0)
model = pipeline("text2text-generation", model="t5-large")
model(prompt, max_length=128, do_sample=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

[{'generated_text': '. Use my phone for this?'}]

In [4]:
from transformers import pipeline
import networkx as nx
prompt = '''### Instruction:
Who is the first person on the moon?

### Response:'''
def generate(prompt):
  output = model(prompt, do_sample=True, min_length=10, max_length=len(prompt)+128)
  out = output[0]['generated_text']
  if '### Response:' in out:
    out = out.split('### Response:')[1]
  if '### Instruction:' in out:
    out = out.split('### Instruction:')[0]
  return out

## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [5]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }
town_people = {
    "Madam Seraphina": "Madam Seraphina is an eccentric and mysterious fortune teller who lives on the outskirts of Phandalin. She is known for her cryptic prophecies and has a reputation for being both revered and feared by the townsfolk. Her dark, flowing robes and a collection of strange trinkets give her an otherworldly appearance. She speaks in riddles and often vanishes without a trace, only to reappear when least expected.",
    "Grizzle the Hermit": "Grizzle is a reclusive and grumpy old dwarf who lives deep in the nearby woods. He was once a master blacksmith, but now he spends his days ranting to himself about the 'cursed' town of Phandalin. Covered in soot and grime, he rarely interacts with others unless they need a weapon forged. When he does speak, it’s in cryptic warnings and grumbles about ancient evils.",
    "Sable Nightshade": "Sable is a charming but sinister alchemist who runs an apothecary in Phandalin. Her shop is filled with strange and exotic concoctions, many of which have questionable effects. Sable has a mysterious past and a penchant for experimenting on unwary visitors. Rumors suggest that she dabbles in dark magic, and some believe she may be behind the recent strange occurrences in town.",
    "Father Graven": "Father Graven is a somber and brooding priest who presides over the Shrine of Luck. Once a fervent believer, he has become disillusioned with the goddess Tymora after a series of personal tragedies. Now, his sermons are filled with despair, and he preaches about the futility of hope. Some townsfolk fear he is losing his sanity, while others believe he is being haunted by a vengeful spirit.",
    "Rictus Smiles": "Rictus is the owner of The Laughing Skull, a macabre and unsettling tavern in Phandalin. He is a tall, gaunt man with a permanent, eerie grin stretched across his face, earning him his name. Rictus delights in telling gruesome tales and has a collection of bizarre artifacts that he claims are cursed. Despite his unsettling demeanor, he is oddly beloved by the locals for his morbid sense of humor and his skill in mixing potent brews.",
    "Mistress Thorn": "Mistress Thorn is a regal and enigmatic woman who is rumored to be a powerful sorceress. She rarely leaves her opulent manor on the edge of town, and when she does, it is often under the cover of night. She has a cold, commanding presence, and the townsfolk whisper that she can control the weather and summon spirits at will. No one knows her true motives, but she is both respected and feared by all who meet her.",
    "The Weaver": "Known only as The Weaver, this enigmatic figure is never seen in public. Instead, they communicate through cryptic messages left on tattered pieces of cloth found around town. Some say The Weaver is a ghost, others a spy, but everyone agrees that their influence is growing. The Weaver seems to know everyone's secrets and often predicts tragic events before they happen, leaving the town in a constant state of unease.",
    "Gretta Greenthumb": "Gretta is a wild-eyed herbalist with a deep connection to the natural world, but she is far from the gentle, nurturing type. She speaks to plants as if they were people and insists that the trees whisper secrets of impending doom. She brews potions that are as likely to cause hallucinations as they are to heal. The townsfolk are wary of her, but they still seek her out when in need, though they never stay long in her presence.",
    "Carnifex the Butcher": "Carnifex is a towering, scarred orc who runs the local butcher shop. He claims to have reformed from a life of violence, but his brutal methods of carving meat suggest otherwise. Carnifex is known for his unnerving habit of whistling cheerful tunes while working, and some say his shop is haunted by the souls of those he’s slain in the past. Despite this, his meats are of exceptional quality, and the townsfolk begrudgingly keep him in business.",
    "Morgana the Seer": "Morgana is a blind, ancient woman who lives in a dilapidated hut on the outskirts of town. Though she cannot see, she claims to have visions of the past, present, and future. Her prophecies are often delivered in cryptic rhymes, and she has a disturbing habit of laughing at seemingly inappropriate moments. The townsfolk both revere and fear her, as her predictions, though often vague, have an unsettling tendency to come true.",
    "Whisper": "Whisper is a mysterious figure who is always shrouded in dark, flowing robes that completely hide their identity. No one knows Whisper’s true face, gender, or even if they are truly human. Whisper communicates only through hushed, barely audible words, as if speaking too loudly would bring doom upon the town. They are a master of secrets, often appearing when least expected to offer cryptic advice or warnings, leaving the townsfolk in a constant state of paranoia.",
}

for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square")
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [6]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [7]:
for name in town_people.keys():
  prompt = "You are {}. {} You just woke up in the town of Phandalin and went out to the Town Square. The following people live in the town: {}. What is your goal for today? Be brief, and use at most 20 words and answer from your perspective.".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Madam Seraphina : You are Mistress Thorn. What is your goal for today? You are Mistress Thorn. You are Mistress Thorn. You are Madam Seraphina. 
Grizzle the Hermit You are Grizzle the Hermit.: Your aim is to be a better.: You are Grizzle the Hermit. You are a grumpy old dwarf who lives deep in the nearby woods..!.: You are Grizzle the Hermit. is. he.  . and. ... ###, and..: Have::
Sable Nightshade  “Sure.”: Sable Nightshade. ### Reaction: No. You are Sable Nightshade.. 
Father Graven  This is a goal. ### Comment: Ask for a response. ### Comment: Tell me your goal. 
Rictus Smiles Only 20 words are required. Take your time to write the answers down.: We are Rictus Smiles. You are The Laughing Skull. 
Mistress Thorn Answer from your perspective. ### Define your class: Men or Women, with your own name.: "Mistress Thorn was right." ### Description: You are Mistress Thorn, a powerful sorceress.:: "Impressor" ### Character: Madam Seraphina. 
The Weaver  Your goal is to get the town back to no

In [8]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    prompt = "You are {}. {} You are planning to: {}. You are currently in {} with the following description: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain."
    action_prompts[name] = prompt

In [9]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  Convert the following paragraph to first person past tense:
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

Madam Seraphina the following paragraph into the following person present tense: ., To the following question: Convert the following paragraph to first person present tense: . They communicate as they whisper.: . They whisper; they whisper. Whisper is often found when least expected. They appear when least expected. They whisper;... and and and## #####] whisper whisper.. You are acting Your character You Are The
Grizzle the Hermit the following sentence into first person past tense: a....! 
Sable Nightshade 
Father Graven  You are a Master.. You know them.... and you are...... and and 
Rictus Smiles :.: or Convert the following paragraph to third person past tense: for most of a night and rarely spoken of. Convert the following sentence to third person present tense: .. to.: ,, by; follow. ### #### ### Paragraph ### ### ### they are who are;,:.: Do ,;
Mistress Thorn  And maybe it helped ### Description: The impressionsor - They leave small, cryptic messages
The Weaver  . The weaver. Yo

Collect the memories people observe.

In [10]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    for name_two in people:
      memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [11]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [12]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

[("False - I think it's a coincidence: . You are Mistress Thorn. You are Mistress Thorn.", 0), ('. You are Mistress Thorn. : You know how to perform this edict. You are Mistress Thorn. Your plans are: : You know how to perform this edict. You know how to perform this edict. You are Mistress Thorn. Your plans are: : You are. . ....: You are are are are. are You are  are ;', 0), (': You are Mistress Thorn. Your plan is: : You are Mistress Thorn.: You Are Mistress Thorn. Your goals are: : You are Mistress Thorn..: You are Mistress Thorn. Your plans are: : You are Mistress Thorn. What is your goal for today? you are Mistress. are . are . are are now..  this:.::', 0), (' You are Madam Seraphina and your intentions are: : You are Madam Seraphina. You are Mistress Thorn. You are Mistress Thorn. Your aim is: : You are Miss Thorn. Why?: : You are Mistress Thorn..; : You are. : are Madam are are are. are You are are are. What are the', 0), ('. You are Mistress Thorn. You have plans for today. : 

# Compress Memories

In [13]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name],
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [14]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name],
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [ ]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += ' You know the following about people: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "What do you do in the next hour? Use at most 10 words to explain."
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    Convert the following paragraph to first person past tense:
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    Convert the following paragraph to a tuple (Action, Object):
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      for name_two in people:
        memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "You are {}. Your plans are: {}. Your memories are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name],
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name],
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


Madam Seraphina Tresendar Manor 9 You are You are You are You are You to use: Convert the following paragraph to first person present tense: her visions seem to be based on the past and present. :: You are; She was: :: You are; She is You are. We are. We are. We are You are. We are. You are You are. You are You are You are You are 
Grizzle the Hermit Tresendar Manor 9  ### Correction: Add the following sentence to the text as shown below: 
Sable Nightshade Tresendar Manor 9 Convert the following paragraph to first person past tense: he has... 
Father Graven Tresendar Manor 9  This person possesses dreams and visions of the future.. She is known for her funny
Rictus Smiles Shrine of Luck 9 s Convert following paragraph to third person present tense: . All the These Things Are. Be . This is: You are. or You:. There are... 
Mistress Thorn Tresendar Manor 9  Convert the following paragraph to first person past tense: She has a power beyond her years and has a cryptic quality. Balance Impre